In [1]:
import numpy as np
import random

In [2]:
def load_movielens(path='./ml-100k'):
    # get movie titles
    movies = {}
    for line in open(path + '/u.item', encoding='latin-1'):
        id, title = line.split('|')[0:2]
        movies[id] = id
    # load data
    prefs = {}
    for line in open(path + '/u.data', encoding='latin-1'):
        user, movieid, rating, ts = line.split('\t')
        prefs.setdefault(user, {})
        prefs[user][movies[movieid]] = float(rating)
    return prefs

In [3]:
prefs = load_movielens()
print(prefs['88'])

{'750': 2.0, '302': 3.0, '321': 1.0, '881': 5.0, '301': 4.0, '315': 4.0, '261': 5.0, '690': 4.0, '904': 5.0, '886': 5.0, '308': 4.0, '311': 5.0, '300': 3.0, '1191': 5.0, '319': 3.0, '313': 3.0, '880': 3.0, '898': 4.0, '354': 5.0, '286': 5.0, '326': 5.0}


In [4]:
def split_data(data, M, k, seed):
    test = []
    train = []
    random.seed(seed)
    for user in data:
        if random.randint(0, M) == k:
            test.append([user, data[user]])
        else:
            train.append([user, data[user]])
    return train, test

In [5]:
train, test = split_data(prefs, 8, 1, 1)

In [6]:
def changement(data):
    x = []
    y = []
    for i in range(len(data)):
        for j in range(1682):
            x.append(data[i][1].get(str(j+1), 0.0))
        y.append(x)
        x = []
    y = np.array(y)
    return y
train = changement(train)
test = changement(test)

In [7]:
def sgd(data_matrix, k, alpha, lam, max_cycles):
    """使用梯度下降法进行矩阵分解。

    Args:
    - data_matrix: mat, 用户物品矩阵
    - k: int, 分解矩阵的参数
    - alpha: float, 学习率
    - lam: float, 正则化参数
    - max_cycles: int, 最大迭代次数

    Returns:
    p,q: mat, 分解后的矩阵
    """
    m, n = np.shape(data_matrix)
    # initiate p & q
    p = np.mat(np.random.random((m, k)))
    q = np.mat(np.random.random((k, n)))

    # start training
    for step in range(max_cycles):
        for i in range(m):
            for j in range(n):
                if data_matrix[i, j] > 0:
                    error = data_matrix[i, j]
                    for r in range(k):
                        error = error - p[i, r] * q[r, j]
                    for r in range(k):
                        p[i, r] = p[i, r] + alpha * (2 * error * q[r, j] - lam * p[i, r])
                        q[r, j] = q[r, j] + alpha * (2 * error * p[i, r] - lam * q[r, j])

        loss = 0.0
        for i in range(m):
            for j in range(n):
                if data_matrix[i, j] > 0:
                    error = 0.0
                    for r in range(k):
                        error = error + p[i, r] * q[r, j]
                    # calculate loss function
                    loss = (data_matrix[i, j] - error) * (data_matrix[i, j] - error)
                    for r in range(k):
                        loss = loss + lam * (p[i, r] * p[i, r] + q[r, j] * q[r, j]) / 2

        if loss < 0.001:
            break
        if step % 10 == 0:
            print("\titer: %d, loss: %f" % (step, loss))
    return p, q

In [ ]:
p, q = sgd(train, 2, 0.003, 0.1, 64)

	iter: 0, loss: 4.923808
	iter: 10, loss: 0.634544
	iter: 20, loss: 0.660026
	iter: 30, loss: 0.661758
	iter: 40, loss: 0.661813
	iter: 50, loss: 0.661179


In [10]:
def prediction(data_matrix, p, q, user):
    """为用户未互动的项打分

    Args:
    - data_matrix: mat, 原始用户物品矩阵
    - p: mat, 分解后的矩阵p
    - q: mat, 分解后的矩阵q
    - user: int, 用户的id

    Returns:
    - predict: list, 推荐列表
    """
    n = np.shape(data_matrix)[1]
    predict = {}
    for j in range(n):
        if data_matrix[user, j] == 0:
            predict[j] = (p[user,] * q[:, j])[0, 0]

    # 按照打分从大到小排序
    return sorted(predict.items(), key=lambda d: d[1], reverse=True)

In [11]:
prediction(train, p, q, 2)

[(1499, 5.034879872712938),
 (21, 4.782307876792727),
 (965, 4.743694062481846),
 (173, 4.664086839900944),
 (264, 4.639784912628163),
 (142, 4.624917371430923),
 (862, 4.620431358903568),
 (691, 4.60844464280019),
 (120, 4.565650206848447),
 (219, 4.5437026863426855),
 (308, 4.543243721044247),
 (495, 4.529560074680197),
 (63, 4.527862755603454),
 (391, 4.520414081520262),
 (1219, 4.511986847746936),
 (68, 4.511203806822724),
 (70, 4.477182101708201),
 (65, 4.465012647952944),
 (1641, 4.463718242831027),
 (165, 4.44685359605036),
 (317, 4.425455948736678),
 (112, 4.418232487723767),
 (209, 4.397547389712154),
 (1191, 4.3939596868015),
 (384, 4.386898173156326),
 (1366, 4.384854868326893),
 (1448, 4.383369468377444),
 (94, 4.379279434402385),
 (160, 4.379141518869469),
 (78, 4.378124086126487),
 (281, 4.372690566871707),
 (124, 4.371857119891068),
 (1130, 4.363085043004563),
 (1442, 4.354926842980995),
 (87, 4.350627649225214),
 (738, 4.350031555193908),
 (27, 4.349350567059639),
 (171